# Limpieza de tablas

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
path_originales = "originales"
os.listdir(path_originales)

['circuits.csv',
 'constructors.csv',
 'constructor_results.csv',
 'constructor_standings.csv',
 'drivers.csv',
 'driver_standings.csv',
 'lap_times.csv',
 'pit_stops.csv',
 'qualifying.csv',
 'races.csv',
 'results.csv',
 'seasons.csv',
 'sprint_results.csv',
 'status.csv']

## Circuitos
Variables y tipo de variable en la tabla.

**Observaciones**:
* La tabla original cuenta con una variable `alt: altitud`, que debería ser entera. Sin embargo, tiene valores nulos identificados como `\\N`, estos serán remplazados por `np.nan` para todas las tablas de la BD

In [3]:
circuits = pd.read_csv(os.path.join(path_originales, 'circuits.csv'))

#Se reemplaza \\N por np.nan
circuits.replace(to_replace='\\N', value=np.nan, inplace=True)

#Se convierte columna de altitud a numérica
circuits['alt'] = circuits['alt'].astype('float')

circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18.0,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109.0,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130.0,http://en.wikipedia.org/wiki/Istanbul_Park


In [4]:
circuits.dtypes

circuitId       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt           float64
url            object
dtype: object

In [5]:
circuits.describe()

,circuitId,lat,lng,alt
count,76.000000,76.000000,76.000000,74.000000
mean,39.355263,33.407770,2.606284,248.189189
std,22.680214,22.958321,64.553438,365.041851
min,1.000000,-37.849700,-118.189000,-7.000000
25%,19.750000,32.417775,-9.250838,19.250000
50%,39.500000,41.061100,4.128885,129.500000
75%,58.250000,47.020150,21.404850,315.500000
max,79.000000,57.265300,144.968000,2227.000000


**Acerca de variables numéricas**:
* Tanto `circuitId`, como `lat`, y `lng` están completas.
* `alt` tiene 2 valores faltantes, mismo que se pueden obtener de `url`

In [6]:
print(f"Valores faltantes en 'alt': {circuits['alt'].isna().sum()}\n")

Valores faltantes en 'alt': 2



In [7]:
#Se obtienen ID's de los circuitos faltantes
circuits.loc[circuits['alt'].isna()]

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
74,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,NaN,http://en.wikipedia.org/wiki/Losail_Internatio...
75,79,miami,Miami International Autodrome,Miami,USA,25.9581,-80.2389,NaN,http://en.wikipedia.org/wiki/Miami_Internation...


La altura de los circuitos faltantes se obtiene de la información geográfica de las ciudades en las que se corren el GP:
* **Losail International Circuit**, cercano a Doha, Catar. **13 m** de altura, [referencia](https://dateandtime.info/es/citycoordinates.php?id=290030#:~:text=Coordenadas%20geogr%C3%A1ficas%20de%20Doha%2C%20Catar%20Latitud%3A%2025%C2%B016%E2%80%B245%E2%80%B3%20N,Altitud%20sobre%20el%20nivel%20del%20mar%3A%2013%20m)
* **Miami International Autodrome**, cercano al Hard Rock Stadium de los Miami Dolphins. **3m** de altura, [referencia](https://elevation.maplogs.com/poi/hard_rock_stadium_don_shula_dr_miami_gardens_fl_usa.416642.html)

In [8]:
# Se completan valores de altitud
circuits.loc[74, 'alt'] = 13
circuits.loc[75, 'alt'] = 3

circuits.loc[circuits['circuitId'].isin([78,79])]

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
74,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,13.0,http://en.wikipedia.org/wiki/Losail_Internatio...
75,79,miami,Miami International Autodrome,Miami,USA,25.9581,-80.2389,3.0,http://en.wikipedia.org/wiki/Miami_Internation...


**Finalmente**:
<br>Se quita la columns `url` de lo que será la tabla de trabajo, y se guarda una copia

In [9]:
circuits.drop(columns='url', inplace=True)
circuits.to_csv('circuits.csv', index=False)

## Constructores

In [10]:
constructors = pd.read_csv(os.path.join(path_originales, 'constructors.csv'))

#Se reemplaza \\N por np.nan
constructors.replace(to_replace='\\N', value=np.nan, inplace=True)

constructors.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [11]:
constructors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   211 non-null    int64 
 1   constructorRef  211 non-null    object
 2   name            211 non-null    object
 3   nationality     211 non-null    object
 4   url             211 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.4+ KB


La tabla de constructores está completa

**Finalmente**:
<br>Se quita la columns `url` de lo que será la tabla de trabajo, y se guarda una copia

In [12]:
constructors.drop(columns='url', inplace=True)
constructors.to_csv('constructors.csv', index=False)

## Pilotos

In [13]:
drivers = pd.read_csv(os.path.join(path_originales, 'drivers.csv'))

#Se reemplaza \\N por np.nan
drivers.replace(to_replace='\\N', value=np.nan, inplace=True)

#Se convierte la fecha de nacimiento a variable tipo date
drivers['dob'] = pd.to_datetime(drivers['dob'])

drivers.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [14]:
drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   driverId     854 non-null    int64         
 1   driverRef    854 non-null    object        
 2   number       51 non-null     object        
 3   code         97 non-null     object        
 4   forename     854 non-null    object        
 5   surname      854 non-null    object        
 6   dob          854 non-null    datetime64[ns]
 7   nationality  854 non-null    object        
 8   url          854 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 60.2+ KB


**Observaciones**:
* A lo largo de la historia han habido 854 pilotos
* Solamente se tiene referencia de número para 51 de ellos
* Solo 97 pilotos cuentan con un código de identificación de 3 letras

No es relevante completar estas columnas debido a que son una referencia única adicional para cada piloto, para dicho propósito se puede utilizar la variable `driverRef`. Dicho eso, se descatan columnas `number`, `code`, y `url`

In [15]:
drivers.drop(columns=['number', 'code', 'url'], inplace=True)
drivers.to_csv('drivers.csv', index=False)